# 读取图像，转成tensor

In [1]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

image_name = 'lenaOriginal.png'
img = Image.open(image_name)
print(type(img))

img = np.array(img)
print(img.shape) #转numpy

img = torch.from_numpy(img.transpose((2, 0, 1)))
print(img.shape)
img = img.numpy().transpose((1, 2, 0))
print(img.shape)

<class 'PIL.PngImagePlugin.PngImageFile'>
(512, 512, 3)
torch.Size([3, 512, 512])
(512, 512, 3)


In [2]:
# 使用torchvision中自带的transforms
image = Image.open(image_name).convert('RGB')
image = transforms.ToTensor()(image).unsqueeze(0)
print(image.shape)

image = image.squeeze(0)
image = transforms.ToPILImage()(image)
print(type(image))

totensor = transforms.ToTensor()
toPIL = transforms.ToPILImage()

torch.Size([1, 3, 512, 512])
<class 'PIL.Image.Image'>


In [3]:
# 图像预处理 normalization
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

normalize = transforms.Normalize(
    mean=norm_mean,
    std=norm_std,
    inplace=False
)

image_nor = normalize(totensor(image))
print(image_nor.mean(), image_nor.std())
image_nor = toPIL(image_nor)
#image.save("lena_normalized.jpg")
image_nor.show()

tensor(-0.4199) tensor(1.0752)


# 图像卷积

In [5]:
# 卷积
x = torch.tensor([[[[3., 0., 1., 2., 7., 4.], 
	              [1., 5., 8., 9., 3., 1.], 
	              [2., 7., 2., 5., 1., 3.], 
	              [0., 1., 3., 1., 7., 8.], 
	              [4., 2., 1., 6., 2., 8.],
	              [2., 4., 5., 2., 3., 9.]]]], requires_grad=False)
w = torch.tensor([[[[1., 0., -1.], 
				  [1., 0., -1.],
				  [1., 0., -1.]]]], requires_grad=True)

out = F.conv2d(x, w, stride=1, padding=0)
print(out)

out1 = F.conv2d(x, w, stride=1, padding=1)
print(out1)

out2 = F.conv2d(x, w, stride=2, padding=0)
print(out2)

tensor([[[[ -5.,  -4.,   0.,   8.],
          [-10.,  -2.,   2.,   3.],
          [  0.,  -2.,  -4.,  -7.],
          [ -3.,  -2.,  -3., -16.]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[ -5.,  -5.,  -6.,  -1.,   6.,  10.],
          [-12.,  -5.,  -4.,   0.,   8.,  11.],
          [-13., -10.,  -2.,   2.,   3.,  11.],
          [-10.,   0.,  -2.,  -4.,  -7.,  10.],
          [ -7.,  -3.,  -2.,  -3., -16.,  12.],
          [ -6.,   0.,  -2.,   1.,  -9.,   5.]]]],
       grad_fn=<ConvolutionBackward0>)
tensor([[[[-5.,  0.],
          [ 0., -4.]]]], grad_fn=<ConvolutionBackward0>)


In [6]:
# 灰度图卷积：边缘检测
grayimage = image.convert('L')
grayimage = totensor(grayimage).unsqueeze(0)
w = torch.tensor([[[[1., 0., -1.], 
				  [2., 0., -2.],
				  [1., 0., -1.]]]])
convv = F.conv2d(grayimage, w, stride=1, padding=0)
convv = convv.squeeze(0)
convv = toPIL(convv)
convv.show()

In [7]:
# padding
convpad = F.conv2d(grayimage, w, stride=1, padding=10)
convpad = convpad.squeeze(0)
convpad = toPIL(convpad)
convpad.show()

In [8]:
# 步长
convstride = F.conv2d(grayimage, w, stride=2, padding=0)
convstride = convstride.squeeze(0)
convstride = toPIL(convstride)
convstride.show()

In [9]:
# 水平线检测
w2 = torch.tensor([[[[1., 2., 1.], 
				  [0., 0., 0.],
				  [-1., -2., -1.]]]])
convh = F.conv2d(grayimage, w2, stride=1, padding=0)
convh = convh.squeeze(0)
convh = toPIL(convh)
convh.show()

In [10]:
# 多通道卷积（RGB）
image = totensor(image).unsqueeze(0)

w = torch.tensor([[[[1., 0., -1.], 
				  [2., 0., -2.],
				  [1., 0., -1.]]]])
w = w.repeat(1,3,1,1)

convv = F.conv2d(image, w, stride=1, padding=0)
convv = convv.squeeze(0)
convv = toPIL(convv)
convv.show()

In [11]:
convpad = F.conv2d(image, w, stride=1, padding=10)
convpad = convpad.squeeze(0)
convpad = toPIL(convpad)
convpad.show()

In [12]:
convstride = F.conv2d(image, w, stride=2, padding=0)
convstride = convstride.squeeze(0)
convstride = toPIL(convstride)
convstride.show()

In [13]:
# 多卷积核
w2 = torch.tensor([[[[1., 2., 1.], 
				  [0., 0., 0.],
				  [-1., -2., -1.]]]])
w2 = w2.repeat(1,3,1,1)
w3 = torch.cat((w,w2),0)
w3 = torch.cat((w3,w2),0)
convh = F.conv2d(image, w3, stride=1, padding=0)
convh = convh.squeeze(0)
convh = toPIL(convh)
convh.show()